In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

# Lecture 03: Enhanced Financial Assistant with Tool Delegation

This notebook demonstrates how to extend a basic agent with specialized tools. The financial assistant from Lecture 01 is enhanced with a `stock_report` tool that internally delegates to the Lecture 02 stock analysis agent.


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent, RunContext
from pydantic_ai.tools import Tool
from common.models import StockReport
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging
from lecture02.agent import analyze_stock

# Initialize logging
setup_logging()

In [ ]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="tool-delegation-lecture03",
    start_message="🚀 Lecture 03 - Tool Delegation Notebook Started",
)

In [ ]:
# Create a tool that delegates to the lecture02 stock analysis agent
async def stock_report(ctx: RunContext, symbol: str) -> StockReport:
    """Get a detailed stock analysis report by delegating to lecture02 agent."""
    return await analyze_stock(symbol)


stock_report_tool = Tool(
    stock_report,
    description="Get a detailed stock analysis report for a stock symbol (e.g., AAPL, TSLA).",
)

# Create enhanced financial assistant with both web_search and stock_report tools
enhanced_financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool, stock_report_tool],  # Now has 2 tools
    system_prompt="""
You are a knowledgeable financial assistant.

When answering questions:
1. Use the web_search tool for general financial information
2. Use the stock_report tool when users ask for detailed analysis of specific stocks
3. Combine information from both sources with your financial knowledge
4. Provide clear, actionable advice
5. Include appropriate disclaimers for investment advice

Keep your responses informative but concise.
""",
)

In [ ]:
# Ask a general financial question (uses web_search)
question = "What is the current price of Bitcoin?"
result = await enhanced_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Ask for detailed stock analysis (uses stock_report tool)
question = "Can you give me a detailed analysis of Apple stock (AAPL)?"
result = await enhanced_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")

In [ ]:
# Try another stock analysis
question = "Give me a comprehensive report on Tesla (TSLA)"
result = await enhanced_financial_agent.run(question)
print(f"Question: {question}\n")
print(f"Response:\n{result.output}")